# IÐN303G Tæknileg kerfi - Sýnidæmi - Stýringar - P

## Inngangur
Markmiðið með þessu sýnidæmi er að kynnast hönnun P stýringa.

Gefið er sjálfvirkt stýrikerfi sem má lýsa með eftirfarandi blokkriti (úr Skilaverkefni 3).

<div>
<img src="https://github.com/rsaemundsson/idn_303_TS_25/blob/main/s3_blokkrit.png?raw=true" alt="Mynd vantar", width=600>


Til stendur að hanna stýringu sem gefur eins hraða svörun við breytingum í óskgildi og hægt er með fasaöryggi upp á a.m.k. 60 gráður og hefur skekkju jafnvægi sem er minni en 0,01 ef breytingin er einingaþrepafall.

Yfirfærsluföll fyrir mismunandi einingar kerfisins eru eftirfarandi:

$$ G_{Stýring}(s)=K_P $$

$$ G_{Loki} (s)=\frac{K_L}{\tau_L s + 1} $$

$$G_{Tankur}(s)=\frac{K_T}{\tau_Ts + 1}$$

$$G_{Nemi}(s)=1$$

  þar sem $K_L = 1/60$, $\tau_L = 1$s, $K_T=2800$ og $\tau_T=250s$.

In [ ]:
!pip install control

In [ ]:
# Hlaða inn þeim forritasöfnum sem þarf að hlaða inn
import numpy as np
import control as ct
import matplotlib.pyplot as plt

# Stuðlar

K_L = 1/60.0
tau_L = 1.0
K_T = 2800.0
tau_T = 250.0

# Hönnunarkröfur

min_phase_margin = 60.0
phase_margin_precision = 1
max_steady_state_error = 0.01

# Breyti K_P þar til ég fæ rétt fasaöryggi

K_P = 1

# Yfirfærsluföll

G_Styring = K_P
G_Loki = ct.tf(K_L,[tau_L, 1])
G_Tankur=ct.tf(K_T,[tau_T, 1])
G_Nemi = 1

G_P = ct.series(G_Loki, G_Tankur)
G_K = ct.series(G_Styring, G_P, G_Nemi)
G_T = ct.feedback(ct.series(G_Styring,G_P), G_Nemi)

# Greining á fasaöryggi

gm, pm, wcg, wcp_init = ct.margin(G_K)

t_s = 1
r_delta = 1

if (pm > min_phase_margin):
  if (np.abs(pm - min_phase_margin)>phase_margin_precision):
    print("Hönnun uppfyllir kröfur um fasaöryggi en svörun gæti verið hraðari")
  else:
    print("Hönnun uppfyllir kröfur um fasaöryggi og svörun gæti ekki verið mikið hraðari")
else:
  print("Hönnun uppfyllir EKKI kröfur um fasaöryggi")

s_info = ct.step_info(G_T)
overshoot = s_info["Overshoot"]

t,y = ct.step_response(G_T)

plt.figure(1)
plt.plot(t_s * t,r_delta * y, label=f"Fasaöryggi: {pm/100:.1%}, Yfirskot: {overshoot/100:.3%}")
plt.grid()
plt.legend()
plt.xlabel('sek')
plt.ylabel('Vatnshæð')

ct.bode(G_K, dB=True)

In [ ]:
#Skoðum skekkju í jafnvægi

G_E = ct.feedback(1,G_K)

s_info = ct.step_info(G_T)
error_ss = r_delta - s_info["SteadyStateValue"]

if (error_ss < max_steady_state_error):
  print("Hönnun uppfyllir kröfur um skekkju í jafnvægi")
else:
  print("Hönnun uppfyllir EKKI kröfur um skekkju í jafnvægi")

t,e = ct.step_response(G_E)
plt.figure(2)
plt.plot(t_s * t,r_delta * e,label=f"Skekkja í jafnvægi: {error_ss:.4f}")
plt.grid()
plt.legend()
plt.xlabel('sek')
plt.ylabel('Skekkja')


In [ ]:
# Skoðum stýrimerki

G_U = ct.feedback(G_Styring,ct.series(G_P,G_Nemi))

t,u = ct.step_response(G_U)
plt.figure(3)
plt.plot(t_s * t,r_delta * u)
plt.grid()
plt.xlabel('sek')
plt.ylabel('Stýrimerki')
